In [140]:
from __future__ import division
import numpy as np
import math as m
from pyomo.environ import *
from pyomo.util.infeasible import log_infeasible_constraints
from coopr.pyomo import *


Recall that you have to create a virtual environment and install the following packages:

pip install numpy 
pip install matplotlib 
conda install -c conda-forge pyomo 
pip install coopr 
conda install -c conda-forge ipopt glpk

In [141]:
# Initialization of concrete model
model = ConcreteModel()

In [142]:
# Problem Set-up
n_flights = 2
F = np.linspace(1,n_flights,n_flights).astype(int)
F = F.astype(str)
for i,f in enumerate(F):
    F[i] = 'f' + f

model.F = Set(initialize=F,ordered=True)

Data = {F[0]: {}, F[1]: {}}
scale = 100000

# This function is just to define the initial/final position X, Y, angle and the Initial Velocity of the aircraft
def InitialData(angle, scale):
    R=100000
    return  {'xI': R*(1-0.5*m.cos(angle))/scale, 'xF': R*(1+0.5*m.cos(angle))/scale,
             'yI': R*(1-0.5*m.sin(angle))/scale, 'yF': R*(1+0.5*m.sin(angle))/scale,
                  'vI': 200/scale, 'chiI': angle, 'chiF': angle}

angles = []
if n_flights==2:
    angles = [m.pi / 2, 0]
else:
    ang_dif = 2*m.pi/n_flights
    for i in range(0,n_flights):
        angles.append(i*ang_dif)

for index, angle in enumerate(angles):
    Data[F[index]] = InitialData(angle, scale)

n_it = 500 # Discrete Samples
dt_extra=10

T = np.linspace(0,n_it,n_it+1).astype(int)

V_Stall={}
V_Max={}
a_Min={}
a_Max={}
chi_Min={}
chi_Max={}
for i,f in enumerate(F):
    V_Stall[F[i]] = 150/scale
    V_Max[F[i]] = 250/scale
    a_Min[F[i]]=-5/scale
    a_Max[F[i]]=5/scale
    chi_Min[F[i]] = Data[F[i]]['chiI'] - 60*m.pi/180
    chi_Max[F[i]] = Data[F[i]]['chiI'] + 60*m.pi/180

r_min=9260/scale

print(Data)

{np.str_('f1'): {'xI': 1.0, 'xF': 1.0, 'yI': 0.5, 'yF': 1.5, 'vI': 0.002, 'chiI': 1.5707963267948966, 'chiF': 1.5707963267948966}, np.str_('f2'): {'xI': 0.5, 'xF': 1.5, 'yI': 1.0, 'yF': 1.0, 'vI': 0.002, 'chiI': 0, 'chiF': 0}}


In [ ]:
# Define the set of Times for the problem

model.T = Set(initialize=T,ordered=True) 
model.n_it = Param(initialize=n_it)
model.dt_extra = Param(initialize=dt_extra)


# model.scale = Param(initialize=scale)

model.T.pprint()

T : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :  501 : {np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(50), np.int64(51), np.int64(52), np.int64(53), np.int64(54), np.int64(55), np.int64(56), np.int64(57), np.int64(58), np.int64(59), np.int64(60), np.int64(61), np.int64(62), np.int6

In [144]:
# Define the Parameters of the problem in the OPT model.

# Initial and Final Positions and Headings
model.xI = Param(model.F, initialize={f: Data[f]['xI'] for f in model.F})
model.xF = Param(model.F, initialize={f: Data[f]['xF'] for f in model.F})

model.yI = Param(model.F, initialize={f: Data[f]['yI'] for f in model.F})
model.yF = Param(model.F, initialize={f: Data[f]['yF'] for f in model.F})

model.chiI = Param(model.F, initialize={f: Data[f]['chiI'] for f in model.F})
model.chiF = Param(model.F, initialize={f: Data[f]['chiF']  for f in model.F})

# Initial Velocities
model.vI = Param(model.F, initialize={f: Data[f]['vI'] for f in model.F})


# Define the flight envelope parameters                                     
model.V_stall = Param(model.F, initialize=V_Stall)
model.V_max = Param(model.F, initialize=V_Max)

model.a_min = Param(model.F, initialize=a_Min)
model.a_max = Param(model.F, initialize=a_Max)

model.chi_min = Param(model.F, initialize=chi_Min)
model.chi_max = Param(model.F, initialize=chi_Max)

model.r_min = Param(initialize=r_min)

model.V_stall.pprint()

V_stall : Size=2, Index=F, Domain=Any, Default=None, Mutable=False
    Key : Value
     f1 : 0.0015
     f2 : 0.0015


In [145]:
# Define the Variables of the problem in the OPT model.
# x,y,v,heading,acceleration, and the final time (function of the flight), step=final time/(number of iterations-1)



model.x = Var(model.F, model.T, within=Reals)

model.y = Var(model.F, model.T, within=Reals)

model.v = Var(model.F, model.T, within=NonNegativeReals)

model.chi = Var(model.F, model.T, within=Reals)

model.a = Var(model.F, model.T, within=Reals)

model.tf = Var(model.F, within=NonNegativeReals)








In [146]:
# Boundary Condition Constraints at t=0 and t=n_it
# Recall that is good practice to define them with a slack tolerance

# initial and final values of velocity or whatever x[t=1]=xiniical x[t=n]=xfinal


# tolerances
epsilon=3/scale
epsilon_angle=3*m.pi/180

# x

# Initial Conditions
def InitialX(model, f):
    return model.x[f,0] == model.xI[f]
model.InitialX_Constraint = Constraint(model.F, rule=InitialX)

# Final Conditions
def FinalX_upp(model, f):
    return model.x[f,model.n_it] <= model.xF[f] + epsilon
model.FinalX_upp_Constraint = Constraint(model.F, rule=FinalX_upp)

def FinalX_low(model, f):
    return model.x[f,model.n_it] >= model.xF[f] - epsilon
model.FinalX_low_Constraint = Constraint(model.F, rule=FinalX_low)  



# y

# Initial Conditions
def InitialY(model, f):
    return model.y[f,0] == model.yI[f]
model.InitialY_Constraint = Constraint(model.F, rule=InitialY)
# Final Conditions
def FinalY_upp(model, f):
    return model.y[f,model.n_it] <= model.yF[f] + epsilon
model.FinalY_upp_Constraint = Constraint(model.F, rule=FinalY_upp)
def FinalY_low(model, f):
    return model.y[f,model.n_it] >= model.yF[f] - epsilon
model.FinalY_low_Constraint = Constraint(model.F, rule=FinalY_low)


# v
# Initial Conditions
def InitialV(model, f):
    return model.v[f,0] == model.vI[f]
model.InitialV_Constraint = Constraint(model.F, rule=InitialV)



# chi
# Initial Conditions
def InitialChi(model, f):
    return model.chi[f,0] == model.chiI[f]
model.InitialChi_Constraint = Constraint(model.F, rule=InitialChi)
# Final Conditions
def FinalChi_upp(model, f):
    return model.chi[f,model.n_it] <= model.chiF[f] + epsilon_angle
model.FinalChi_upp_Constraint = Constraint(model.F, rule=FinalChi_upp)
def FinalChi_low(model, f):
    return model.chi[f,model.n_it] >= model.chiF[f] - epsilon_angle
model.FinalChi_low_Constraint = Constraint(model.F, rule=FinalChi_low)



In [148]:
# Constraints associated with the dynamics of the system

# this are the discretized eq 4,5,6 for t in T

# Eq(4)
def DynamicX(model, f, t):
    if t < model.T(model.n_it):
      model.t_step = model.tf[f]/(model.n_it-1)
      return model.x[f,t+1] == model.x[f,t] + model.v[f,t]*m.cos(model.chi[f,t])*model.t_step 
    else:
      return Constraint.Skip
    
model.DynamicX_Constraint = Constraint(model.F, model.T, rule=DynamicX)

# Eq(5)
def DynamicY(model, f, t):
    if t < model.T(model.n_it):
        model.t_step = model.tf[f]/(model.n_it-1)
        return model.y[f,t+1] == model.y[f,t] + model.v[f,t]*m.sin(model.chi[f,t])*model.t_step 
    else:
        return Constraint.Skip
model.DynamicY_Constraint = Constraint(model.F, model.T, rule=DynamicY)

# Eq(6)
def DynamicV(model, f, t):
    if t < model.T(model.n_it):
        model.t_step = model.tf[f]/(model.n_it-1)
        return model.v[f,t+1] == model.v[f,t] + model.a[f,t]*model.t_step 
    else:
        return Constraint.Skip
model.DynamicV_Constraint = Constraint(model.F, model.T, rule=DynamicV)





(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
ERROR: Rule failed when generating expression for Constraint
DynamicX_Constraint with index (np.str_('f1'), np.int64(0)): TypeError:
'OrderedScalarSet' object is not callable
ERROR: Constructing component 'DynamicX_Constraint' from data=None failed:
        TypeError: 'OrderedScalarSet' object is not callable


TypeError: 'OrderedScalarSet' object is not callable

In [ ]:
# Flight Envelope Constraints (Inequalities)


# max speed, minum speed, etc

# Eq(7) - Heading Lower Bound
def Heading_LB(model, f, t):
    return model.chi[f,t] >= model.chi_min[f]
model.Heading_LB_Constraint = Constraint(model.F, model.T, rule=Heading_LB)
# Eq(7) - Heading Upper Bound
def Heading_UB(model, f, t):
    return model.chi[f,t] <= model.chi_max[f]
model.Heading_UB_Constraint = Constraint(model.F, model.T, rule=Heading_UB)


# Eq(8) - Velocity Lower Bound
def Velocity_LB(model, f, t):
    return model.v[f,t] >= model.V_stall[f]
model.Velocity_LB_Constraint = Constraint(model.F, model.T, rule=Velocity_LB)
# Eq(8) - Velocity Upper Bound
def Velocity_UB(model, f, t):
    return model.v[f,t] <= model.V_max[f]
model.Velocity_UB_Constraint = Constraint(model.F, model.T, rule=Velocity_UB)


# Eq(9) - Acceleration Lower Bound
def Acceleration_LB(model, f, t):
    return model.a[f,t] >= model.a_min[f]
model.Acceleration_LB_Constraint = Constraint(model.F, model.T, rule=Acceleration_LB)
# Eq(9) - Acceleration Upper Bound
def Acceleration_UB(model, f, t):
    return model.a[f,t] <= model.a_max[f]
model.Acceleration_UB_Constraint = Constraint(model.F, model.T, rule=Acceleration_UB)









In [ ]:
# Conflict Avoidance Constraint



# Eq (10) and (11)
def minimum_separation(model, f, t):

    for fp in model.F:
        if f != fp:
            return (model.x[f,t] - model.x[fp,t])**2 + (model.y[f,t] - model.y[fp,t])**2 >= model.r_min**2
    return Constraint.Skip

model.minimum_separation_Constraint = Constraint(model.F, model.T, rule=minimum_separation)

   

In [ ]:
# Objective Function

def heading_objective(model):
    return sum((model.chi[f,t]- model.chi[f,t-1])**2 for f in model.F for t in model.T if t>0)

model.Objective = Objective(rule=heading_objective, sense=minimize)



In [ ]:
# Initial Guess to feed the Solver.
# Recall that all the variables must be initialized with a value before calling the Solver.

#model.x[f,t].value = 
#model.y[f,t].value = 
#model.v[f,t].value = 
#model.chi[f,t].value =
#model.a[f,t].value =
#model.tf[f].value = 

# we need to provide a soution for the solver to start with, it can be equal cero, and you are far from potimal, or something already close to what the sol should be

for f in model.F:
    for t in model.T:
        


In [76]:
# Solver
opt = SolverFactory("ipopt")
opt.options['print_level'] = 5
#opt.options['halt_on_ampl_error'] = 'yes'
#opt.options['tol'] = 1e-20
#opt.options['constr_viol_tol'] = 1e-20
#opt.options['acceptable_constr_viol_tol'] = 1e-15
opt.options['warm_start_init_point']='yes'
results = opt.solve(model,tee=True)
model.obj.display()
print(results)

ValueError: No variables appear in the Pyomo model constraints or objective. This is not supported by the NL file interface

In [ ]:
# Plots
import matplotlib;
import matplotlib.pyplot as plt


In [ ]:
# Animation Function
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

fig = plt.figure()
ax = plt.axes()
x0, y0, dx, dy = ax.get_position().bounds
maxd = max(dx, dy)
width = 6 * maxd / dx
height = 6 * maxd / dy
fig.set_size_inches((width, height))

def animation(ii):
        fig.clear()
        plt.xlim([50000,150000])
        plt.ylim([50000,150000])
        plt.yticks(rotation=55)
        plt.xlabel('$x \; (m)$')
        plt.ylabel('$y \; (m)$')
        plt.title('Aircraft Trajectory')
        plt.grid()

        color = ['red', 'blue', 'green', 'magenta', 'black', 'purple', 'orange', 'cyan']

        for j in range(0,n_flights):
            ac = plt.scatter(x[j, ii], y[j, ii], color=color[j], marker='.', facecolors='none')
            circ = plt.Circle((x[j, ii], y[j, ii]), radius=9260 / 2, color=color[j], fill=False)
            plt.gca().add_patch(circ)
        plt.legend((F),loc=1)

animator = FuncAnimation(fig, animation, interval = 1/1000, frames=500)
plt.legend(F)